In [ ]:
# dependencies

import gzip
from pickle import dump, load

import numpy as np
import pandas as pd

from switch import mcmc
from switch.models import tanh

In [ ]:
# configure algorithm

n_samples = 100000
n_init = 10000
n_thin = 10
init_scale_h = 2
n_dc_splits = 4
pr_portkey_thi = 1e-3
seed = 0

# configure prior

prior_shape_jumps = 1
prior_rate_jumps = 48

In [ ]:
# import data

f109 = pd.read_csv('paper/f109.csv', sep=',', parse_dates=['date']).iloc[:, 1:]
f109

In [ ]:
# select 2009 data

t = f109.cumTime.values[f109.date.dt.year == 2009]
vt = f109.centerE.values[f109.date.dt.year == 2009]

In [ ]:
# set up sampler

samples = []
sampler = mcmc.sample_posterior(
    t,
    vt,
    2,
    prior_shape_jumps,
    prior_rate_jumps,
    tanh.mod,
    np.random.default_rng(seed),
    init_scale_h=init_scale_h,
    n_init=n_init,
    n_dc_splits=n_dc_splits,
    pr_portkey_thi=pr_portkey_thi)

In [ ]:
# simulate markov chain

for _ in range(n_samples):
    s = next(sampler)
    if not _ % n_thin:
        samples.append(s[:3])

In [ ]:
# dump to disc

state = {k: v for k, v in sampler.gi_frame.f_locals.items() if k in
         ('thi', 'lam', 'h', 'z', 't', 'vt', 'hyper_lam', 'thi_samplers', 'node_sampler', 'ctrl', 'ome')}
with gzip.open('paper/f109_mcmc_state.pkl', 'wb') as f:
    dump(state, f)
with gzip.open('paper/f109_mcmc_samples.pkl', 'wb') as f:
    dump(samples, f)

In [ ]:
# to restart from saved state

# with gzip.open('paper/f109_mcmc_state.pkl', 'rb') as f:
#     state = load(f)
# with gzip.open('paper/f109_mcmc_samples.pkl', 'rb') as f:
#     samples = load(f)

# sampler = mcmc.resume_sampling(**state, mod=tanh.mod)